In [1]:
!pwd

/Users/sofiepelvig/apartments/notebooks


In [2]:
%cd ..

/Users/sofiepelvig/apartments


In [3]:
from ipyleaflet import (Map, basemaps, basemap_to_tiles, 
                        GeoData, Marker, MarkerCluster)

import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path

In [4]:
df = pd.read_csv("assets/data/bolig.csv.gzip", compression="gzip", parse_dates=["soldDate"])
df.tail(3)

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
99208,0,"Hørsholmsgade 8, 4. tv",2200,11947874,1992-01-06 23:00:00+00:00,3,Alm. Salg,225431.580,2.0,53,...,0.0,DA7226E2-F5C9-43D1-AEE6-F928CE45212E,55.690900,12.543945,101,260348,København N,1.0,True,6019808
99209,0,"Peter Fabers Gade 41, 2. th",2200,250000,1992-01-03 23:00:00+00:00,3,Alm. Salg,5000.000,2.0,50,...,0.0,D9388837-4672-4B9A-A1FE-C65BB6397DD8,55.691830,12.556379,101,436128,København N,NaN,True,146895
99210,0,"Øresundsvej 108, 1. tv",2300,351000,1992-01-02 23:00:00+00:00,3,Alm. Salg,5754.098,2.0,61,...,0.0,B18BD911-7FF3-4977-A46D-D324E247A543,55.661255,12.621660,101,661318,København S,NaN,True,133305


In [5]:
alm_salg_df = df[df['saleType'].eq('Alm. Salg')]

#Sorting apartments away that where sold for under 500.000 and above 9.000.000
alm_salg_df = alm_salg_df[(alm_salg_df['price'] > 5e5) & (alm_salg_df['price'] < 9e6)]

In [6]:
estates_sold_in_2008 = df[df['soldDate'].dt.year == 2008]
estates_sold_in_2008

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
12606,339746,"Borups Alle 128, 5. 3",2000,1220000,2008-12-21 23:00:00+00:00,3,Alm. Salg,18769.230,2.0,65,...,-5.791506,F2FF3F4D-D5A9-4F02-9857-DCFEC48666B5,55.694000,12.532352,147,238541,Frederiksberg,NaN,True,182077
12607,299376,"Godthåbsvænget 13, 1. tv",2000,1900000,2008-12-20 23:00:00+00:00,3,Alm. Salg,18811.880,3.0,101,...,-4.761905,581C9AE2-7A4F-4E4F-B173-91973BEDA848,55.690754,12.519732,147,150652,Frederiksberg,NaN,True,177461
12608,0,"Marielystvej 2, st. tv",2000,1075000,2008-12-20 23:00:00+00:00,3,Alm. Salg,16044.776,2.0,67,...,0.000000,2197EC7C-45C4-4985-88A0-5CA21791A534,55.678215,12.500235,147,75723,Frederiksberg,NaN,True,192996
12609,0,"Godthåbsvænget 15, 5. tv",2000,1700000,2008-12-17 23:00:00+00:00,3,Auktion,17346.940,3.0,98,...,0.000000,2F39214C-37CB-43AB-BD13-49DBB7907457,55.690845,12.519802,147,150865,Frederiksberg,NaN,True,177483
12610,0,"Madvigs Alle 4, 2. th",1829,2925000,2008-12-14 23:00:00+00:00,3,Alm. Salg,26590.908,4.0,110,...,0.000000,8DC4BEBA-6447-4958-A0C1-2140E06A99DB,55.674652,12.540952,147,74263,Frederiksberg C,NaN,True,190089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81873,161542,"Strandboulevarden 38, 5. th",2100,4250000,2008-01-02 23:00:00+00:00,3,Alm. Salg,32442.748,5.0,131,...,-5.239688,E822AD8C-EAD5-48A3-AA5D-4DBB91F91F22,55.700066,12.590268,101,709116,København Ø,NaN,True,157978
81874,0,"Havneholmen 86, 6. tv",1561,5595000,2008-01-02 23:00:00+00:00,3,Alm. Salg,53285.715,3.0,105,...,0.000000,9599A5E2-1D9F-4573-81E8-67754B7D8776,55.661015,12.564528,101,6079,København V,NaN,True,164556
81875,0,"Gimles Allé 8, 1",2300,2625000,2008-01-01 23:00:00+00:00,3,Alm. Salg,31250.000,3.0,84,...,0.000000,258A90E7-A53F-44B8-8483-0E0D86B90ECB,55.651073,12.610314,101,180816,København S,3.0,True,6011653
81876,0,"Gimles Allé 8, st",2300,2625000,2008-01-01 23:00:00+00:00,3,Alm. Salg,29829.545,3.0,88,...,0.000000,F06F31C7-C7C1-45E7-9BCA-58B5BE64FE37,55.651073,12.610314,101,180816,København S,3.0,True,6011653


In [7]:
estates_sold_in_2009 = df[df['soldDate'].dt.year == 2009]
estates_sold_in_2009

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
12090,0,"Sylows Alle 23, 1. mf",2000,3500000,2009-12-28 23:00:00+00:00,3,Alm. Salg,28225.807,4.0,124,...,0.000000,A8BB6ACE-73B7-45D9-B81B-46BE012199C9,55.680054,12.530478,147,258230,Frederiksberg,NaN,True,192776
12091,0,"Rådmand Steins Alle 29, 1. tv",2000,1625000,2009-12-25 23:00:00+00:00,3,Alm. Salg,16752.578,4.0,97,...,0.000000,FCC9D576-D5F3-486A-A272-FCC903FCDF31,55.672848,12.496574,147,109555,Frederiksberg,NaN,True,185024
12092,441470,"Kochsvej 21, st. tv",1812,3900000,2009-12-21 23:00:00+00:00,3,Alm. Salg,36111.110,4.0,108,...,-2.377972,9B7EC753-F27C-4283-9EC1-4BEDE17295AF,55.671955,12.537684,147,67518,Frederiksberg C,NaN,True,193682
12093,0,"Sprogøvej 11, 4. tv",2000,1000000,2009-12-21 23:00:00+00:00,3,Andet,10000.000,4.0,100,...,0.000000,5DA6057D-3BB6-4C73-AB50-AF5FCB8CA61A,55.682960,12.527584,147,219407,Frederiksberg,NaN,True,176744
12094,373176,"Johannes V. Jensens Alle 44, 1. tv",2000,3998000,2009-12-21 23:00:00+00:00,3,Alm. Salg,19990.000,5.0,200,...,0.000000,9B428F08-648D-4B56-8BC2-EDF8F035F8C2,55.671670,12.498742,147,134282,Frederiksberg,NaN,True,194104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81063,292651,"Edvard Falcks Gade 4, 4. th",1569,2600000,2009-01-05 23:00:00+00:00,3,Alm. Salg,27956.988,3.0,93,...,-10.220995,B1A71E45-8DAF-45F6-A8E5-A38CFC68F99A,55.671146,12.570691,101,431720,København V,NaN,True,173315
81064,348072,"Vibekegade 22, st. tv",2100,1880000,2009-01-05 23:00:00+00:00,3,Alm. Salg,26857.143,3.0,70,...,-5.527638,4DCC2032-2734-4337-9C94-32C59C9B19A8,55.705124,12.555851,101,942236,København Ø,NaN,True,156586
81065,0,"Sølvgade 23, 3. mf",1307,1615500,2009-01-04 23:00:00+00:00,3,Fam. Salg,25242.188,2.0,64,...,0.000000,AEF80158-DA7F-411C-9DF9-4D92B60E3BF2,55.685840,12.583649,101,853578,København K,NaN,True,128598
81066,0,"A.C. Meyers Vænge 3, st. tv",2450,1500000,2009-01-03 23:00:00+00:00,3,Fam. Salg,10638.298,5.0,141,...,0.000000,BC747B12-ED34-41BB-B700-0783426DB333,55.652596,12.543877,101,770150,København SV,NaN,True,120527


In [10]:
%%time

# SAMPLE =20_000
YEAR = 2009  # greater or equal to <=
markers = (
    alm_salg_df
     .loc[lambda d: d["soldDate"].dt.year.ge(YEAR)]
     .assign(location=lambda d: d[["latitude", "longitude"]].agg(
         lambda d: (d["latitude"], d["longitude"]) ,axis=1),  # (latitude, longitude)
            title = lambda d: d[["address", "zipCode"]].agg(
         lambda d: f'{d["address"]}, {d["zipCode"]}' ,axis=1), # title will be address
          alt = (lambda d: d[["price", "size", "rooms" , "buildYear"]].agg(
         lambda d: f'price: {d["price"]}, size: {d["size"]}, rooms: {d["rooms"]}, build year: {d["buildYear"]}' ,axis=1))  # title will be address
    )
    .loc[:, ["location", "title", "alt"]]
    # .sample(SAMPLE)
    .agg(lambda d: Marker(location=d["location"], title=d["title"], alt=d["alt"]), axis=1)
)

CPU times: user 17.9 s, sys: 2.27 s, total: 20.1 s
Wall time: 22.2 s


In [11]:
markers.shape[0]

56335

In [15]:
#creating the map and centering it at sofie
# 55.692205626007144, 12.545636686596445
m = Map(center=(55.69220562, 12.5456366), zoom=15)

In [18]:
#adding the markers as a cluster 
marker_cluster = MarkerCluster(
    markers=list(markers)
)

In [20]:
print(markers)

0        Marker(alt='price: 5400000.0, size: 98.0, room...
1        Marker(alt='price: 4195000.0, size: 80.0, room...
2        Marker(alt='price: 3475000.0, size: 61.0, room...
4        Marker(alt='price: 3744000.0, size: 107.0, roo...
6        Marker(alt='price: 3150000.0, size: 71.0, room...
                               ...                        
81061    Marker(alt='price: 1659000.0, size: 60.0, room...
81062    Marker(alt='price: 1900000.0, size: 100.0, roo...
81063    Marker(alt='price: 2600000.0, size: 93.0, room...
81064    Marker(alt='price: 1880000.0, size: 70.0, room...
81067    Marker(alt='price: 1400000.0, size: 43.0, room...
Length: 66625, dtype: object


In [8]:
# Add the GeoData layer to the map
m.add_layer(marker_cluster)

# Display the map
m

NameError: name 'm' is not defined

In [ ]:
1 + 1

In [ ]:
markers.isna().sum()